In [1]:
import os
import numpy as np
import pandas as pd 
from krx_fr.preprocess import tools, scaler
from krx_fr.cluster import evaluate
from krx_fr.cluster.optimize import Optimizer
from krx_fr.cluster.kmeans import MyKmeans

In [2]:
model = "3호모델드랍down"

# 윈도우는 Y로 고정
window_size = "Y"

# 어떻게 윈도우를 밀건지
freq = "Y"

# 클러스터 수
num_k = 3

# 사용할 스케일러
sca = scaler.new_rank
sc = "new_rank"

kospi_listed = tools.load_json("..\\final_data\\코스피_구성종목.json")
kosdaq_listed = tools.load_json("..\\final_data\\코스닥_구성종목.json")
together_listed = tools.aggregate_listed_code(kospi_listed, kosdaq_listed)

# 모델명
model_name = "{}_{}_k{}".format(model, freq, num_k)
model_name

'3호모델드랍down_Y_k3'

In [3]:
# 퐅더에서 row_data 불러오기
kind_data = tools.read_data_from_folder("C:\\Jehyeon\\2022_Winter_KRX\\KRX\\final_data\\KIND\\")
financial_data = tools.read_data_from_folder("C:\\Jehyeon\\2022_Winter_KRX\\KRX\\final_data\\재무\\")

# 최대주주비율은 1달 단위의 pct_change
financial_data2 = tools.dict_data_calculate_pct(financial_data.copy(), name="최대주주비율", return_original=True)

# label_data 불러오기
label_data_war = pd.read_csv("C:\\Jehyeon\\2022_Winter_KRX\\KRX\\final_data\\label\\투자주의환기종목지정카운트.csv", index_col=0)
label_data_man= pd.read_csv("C:\\Jehyeon\\2022_Winter_KRX\\KRX\\final_data\\label\\관리종목카운트.csv", index_col=0)
label_data_emb = pd.read_csv("C:\\Jehyeon\\2022_Winter_KRX\\KRX\\final_data\\label\\횡령배임혐의발생카운트.csv", index_col=0)

In [4]:
financial_data2["결산기간_주가변동률"] = financial_data2["결산기간_주가변동률"].fillna(0)

In [5]:
# 레이블 데이터 shift시켜주기
label_data_man_shift = tools.shift_label_data(label_data_man.copy(),shift_size='Y')
label_data_war_shift = tools.shift_label_data(label_data_war.copy(),shift_size='Y')
label_data_emb_shift = tools.shift_label_data(label_data_emb.copy(),shift_size='Y')

label_data_pp1 = tools.dict_data_preprocess_upgrade(label_data_man_shift.copy(), np.sum, "Y", rolling_size=freq)
label_data_pp_man = tools.dict_data_scale(label_data_pp1.copy(), scaler.zero_one)

label_data_pp2 = tools.dict_data_preprocess_upgrade(label_data_war_shift.copy(), np.sum, "Y", rolling_size=freq)
label_data_pp_war = tools.dict_data_scale(label_data_pp2.copy(), scaler.zero_one)

label_data_pp3 = tools.dict_data_preprocess_upgrade(label_data_emb_shift.copy(), np.sum, "Y", rolling_size=freq)
label_data_pp_emb = tools.dict_data_scale(label_data_pp3.copy(), scaler.zero_one)

In [6]:
# 데이터처리 sum / mean
kind_data2 = tools.dict_data_preprocess_upgrade(kind_data.copy(), function=np.sum, window_size=window_size,rolling_size=freq, fillna=True)
financial_data3 = tools.dict_data_preprocess_upgrade(financial_data2.copy(), function=np.mean, window_size=window_size, rolling_size=freq,fillna=False)

In [7]:
# 재무데이터와 공시데이터 합치기
concat_data = tools.dict_data_merge(kind_data2.copy(),financial_data3.copy(), dropna=False)

In [8]:
# 상장사만 분리한다
list_data = tools.dict_data_match_listed_upgrade(concat_data.copy(), together_listed.copy(), rolling_size=freq)

In [9]:
# 컬럼을 더할 경우
concat_together1 = tools.dict_data_plus(list_data.copy(),
                                  plus_one="BW금액"
                                 ,plus_two="CB금액", return_origianl=False)

#
concat_together1["2012-01"].columns

Index(['BW금액+CB금액', '경영권분쟁소송', '불성실공시법인지정카운트', '상호변경카운트', '업종변경카운트', '유상증자금액',
       '조회공시요구카운트', '최대주주변경을수반하는주식담보제공금액', '최대주주변경을수반하는주식양수도금액', '최대주주변경카운트',
       '타법인주식및출자증권취득결정금액', '특수관계인에대한자금대여금액', '결산기간_거래량증가', '결산기간_주가변동률',
       '당기순이익', '매출채권', '매출총이익', '영업이익', '영업현금흐름', '차입금의존도', '총자본', '최대주주비율',
       '판매 및 관리비', '최대주주비율_변동성'],
      dtype='object')

In [10]:
concat_together2 = tools.dict_data_plus(concat_together1, 
                                  plus_one="BW금액+CB금액"
                                 ,plus_two="유상증자금액", return_origianl=False)
#concat_kospi2 = tools.dict_data_plus(concat_kospi1, 
#                                  plus_one="상호변경카운트_전체시장"
#                                 ,plus_two="업종변경카운트_전체시장", return_origianl=False)
#concat_kosdaq2 = tools.dict_data_plus(concat_kosdaq1, 
#                                  plus_one="상호변경카운트_전체시장"
#                                 ,plus_two="업종변경카운트_전체시장", return_origianl=False)
concat_together2["2012-01"].columns

Index(['BW금액+CB금액+유상증자금액', '경영권분쟁소송', '불성실공시법인지정카운트', '상호변경카운트', '업종변경카운트',
       '조회공시요구카운트', '최대주주변경을수반하는주식담보제공금액', '최대주주변경을수반하는주식양수도금액', '최대주주변경카운트',
       '타법인주식및출자증권취득결정금액', '특수관계인에대한자금대여금액', '결산기간_거래량증가', '결산기간_주가변동률',
       '당기순이익', '매출채권', '매출총이익', '영업이익', '영업현금흐름', '차입금의존도', '총자본', '최대주주비율',
       '판매 및 관리비', '최대주주비율_변동성'],
      dtype='object')

In [11]:
concat_together3 = tools.dict_data_plus(concat_together2, 
                                  plus_one="최대주주변경을수반하는주식담보제공금액"
                                 ,plus_two="최대주주변경을수반하는주식양수도금액", return_origianl=False)
#concat_kospi3 = tools.dict_data_plus(concat_kospi2, 
#                                  plus_one="최대주주변경을수반하는주식담보제공금액_전체시장"
#                                 ,plus_two="최대주주변경을수반하는주식양수도금액_전체시장", return_origianl=False)
#concat_kosdaq3 = tools.dict_data_plus(concat_kosdaq2, 
#                                  plus_one="최대주주변경을수반하는주식담보제공금액_전체시장"
#                                 ,plus_two="최대주주변경을수반하는주식양수도금액_전체시장", return_origianl=False)
#
concat_together3["2012-01"].columns

Index(['최대주주변경을수반하는주식담보제공금액+최대주주변경을수반하는주식양수도금액', 'BW금액+CB금액+유상증자금액', '경영권분쟁소송',
       '불성실공시법인지정카운트', '상호변경카운트', '업종변경카운트', '조회공시요구카운트', '최대주주변경카운트',
       '타법인주식및출자증권취득결정금액', '특수관계인에대한자금대여금액', '결산기간_거래량증가', '결산기간_주가변동률',
       '당기순이익', '매출채권', '매출총이익', '영업이익', '영업현금흐름', '차입금의존도', '총자본', '최대주주비율',
       '판매 및 관리비', '최대주주비율_변동성'],
      dtype='object')

In [12]:
concat_together4 = tools.dict_data_plus(concat_together3, 
                                  plus_one="타법인주식및출자증권취득결정금액"
                                 ,plus_two="특수관계인에대한자금대여금액", return_origianl=False)
#concat_kospi4 = tools.dict_data_plus(concat_kospi3, 
#                                  plus_one="자기주식처분결정금액_전체시장"
#                                 ,plus_two="특수관계인에대한자금대여금액_전체시장", return_origianl=False)
#concat_kosdaq4 = tools.dict_data_plus(concat_kosdaq3, 
#                                  plus_one="자기주식처분결정금액_전체시장"
#                                 ,plus_two="특수관계인에대한자금대여금액_전체시장", return_origianl=False)
concat_together4["2012-01"].columns

Index(['타법인주식및출자증권취득결정금액+특수관계인에대한자금대여금액',
       '최대주주변경을수반하는주식담보제공금액+최대주주변경을수반하는주식양수도금액', 'BW금액+CB금액+유상증자금액', '경영권분쟁소송',
       '불성실공시법인지정카운트', '상호변경카운트', '업종변경카운트', '조회공시요구카운트', '최대주주변경카운트',
       '결산기간_거래량증가', '결산기간_주가변동률', '당기순이익', '매출채권', '매출총이익', '영업이익', '영업현금흐름',
       '차입금의존도', '총자본', '최대주주비율', '판매 및 관리비', '최대주주비율_변동성'],
      dtype='object')

In [13]:
concat_together5 = tools.dict_data_plus(concat_together4,
                                  plus_one="상호변경카운트"
                                 ,plus_two="업종변경카운트", return_origianl=False)
#concat_kospi5 = tools.dict_data_plus(concat_kospi4,
#                                  plus_one="불성실공시법인지정카운트_전체시장"
#                                 ,plus_two="조회공시요구카운트_전체시장", return_origianl=False)
#concat_kosdaq5 = tools.dict_data_plus(concat_kosdaq4,
#                                  plus_one="불성실공시법인지정카운트_전체시장"
#                                 ,plus_two="조회공시요구카운트_전체시장", return_origianl=False)
concat_together5["2012-01"].columns

Index(['상호변경카운트+업종변경카운트', '타법인주식및출자증권취득결정금액+특수관계인에대한자금대여금액',
       '최대주주변경을수반하는주식담보제공금액+최대주주변경을수반하는주식양수도금액', 'BW금액+CB금액+유상증자금액', '경영권분쟁소송',
       '불성실공시법인지정카운트', '조회공시요구카운트', '최대주주변경카운트', '결산기간_거래량증가', '결산기간_주가변동률',
       '당기순이익', '매출채권', '매출총이익', '영업이익', '영업현금흐름', '차입금의존도', '총자본', '최대주주비율',
       '판매 및 관리비', '최대주주비율_변동성'],
      dtype='object')

In [14]:
concat_together6 = tools.dict_data_plus(concat_together5,
                                  plus_one="경영권분쟁소송"
                                 ,plus_two="불성실공시법인지정카운트", return_origianl=False)
#concat_kospi7 = tools.dict_data_plus(concat_kospi6,
#                                  plus_one="BW금액_전체시장+CB금액_전체시장"
#                                 ,plus_two="유상증자금액_전체시장", return_origianl=False)
#concat_kosdaq7 = tools.dict_data_plus(concat_kosdaq6,
#                                  plus_one="BW금액_전체시장+CB금액_전체시장"
#                                 ,plus_two="유상증자금액_전체시장", return_origianl=False)
concat_together6["2012-01"].columns

Index(['경영권분쟁소송+불성실공시법인지정카운트', '상호변경카운트+업종변경카운트',
       '타법인주식및출자증권취득결정금액+특수관계인에대한자금대여금액',
       '최대주주변경을수반하는주식담보제공금액+최대주주변경을수반하는주식양수도금액', 'BW금액+CB금액+유상증자금액',
       '조회공시요구카운트', '최대주주변경카운트', '결산기간_거래량증가', '결산기간_주가변동률', '당기순이익', '매출채권',
       '매출총이익', '영업이익', '영업현금흐름', '차입금의존도', '총자본', '최대주주비율', '판매 및 관리비',
       '최대주주비율_변동성'],
      dtype='object')

In [15]:
concat_together7 = tools.dict_data_plus(concat_together6,
                                  plus_one="경영권분쟁소송+불성실공시법인지정카운트"
                                 ,plus_two="조회공시요구카운트", return_origianl=False)
#concat_kospi8 = tools.dict_data_plus(concat_kospi7,
#                                  plus_one="경영권분쟁소송_전체시장+최대주주변경카운트_전체시장"
#                                 ,plus_two="상호변경카운트_전체시장+업종변경카운트_전체시장", return_origianl=False)
#concat_kosdaq8 = tools.dict_data_plus(concat_kosdaq7,
#                                  plus_one="경영권분쟁소송_전체시장+최대주주변경카운트_전체시장"
#                                 ,plus_two="상호변경카운트_전체시장+업종변경카운트_전체시장", return_origianl=False)
concat_together7["2012-01"].columns

Index(['경영권분쟁소송+불성실공시법인지정카운트+조회공시요구카운트', '상호변경카운트+업종변경카운트',
       '타법인주식및출자증권취득결정금액+특수관계인에대한자금대여금액',
       '최대주주변경을수반하는주식담보제공금액+최대주주변경을수반하는주식양수도금액', 'BW금액+CB금액+유상증자금액',
       '최대주주변경카운트', '결산기간_거래량증가', '결산기간_주가변동률', '당기순이익', '매출채권', '매출총이익',
       '영업이익', '영업현금흐름', '차입금의존도', '총자본', '최대주주비율', '판매 및 관리비', '최대주주비율_변동성'],
      dtype='object')

In [16]:
drop_col_top_y = ["매출총이익","총자본","영업이익","매출채권"] # q와 동일
#drop_col_top_q = ["당기순이익/총자본","영업이익/총자본","총자본","매출총이익/총자본"]

drop_col_down_y = ["최대주주변경카운트","최대주주변경을수반하는주식담보제공금액+최대주주변경을수반하는주식양수도금액","타법인주식및출자증권취득결정금액+특수관계인에대한자금대여금액","상호변경카운트+업종변경카운트"]

divide_data2 = tools.dict_data_drop(concat_together7.copy(), drop_col=drop_col_down_y, dropna=False)

In [17]:
drop_togehter = tools.dict_data_drop(divide_data2.copy(), dropna=True)
scaled_together = tools.dict_data_scale(drop_togehter.copy(), scaler=sca)

scaled_together["2012-01"].columns

Index(['경영권분쟁소송+불성실공시법인지정카운트+조회공시요구카운트', 'BW금액+CB금액+유상증자금액', '결산기간_거래량증가',
       '결산기간_주가변동률', '당기순이익', '매출채권', '매출총이익', '영업이익', '영업현금흐름', '차입금의존도',
       '총자본', '최대주주비율', '판매 및 관리비', '최대주주비율_변동성'],
      dtype='object')

In [18]:
model_save_dict = evaluate.run_iter_kmeans(num_k, max_sample=100, raw_data=scaled_together.copy())
model_save_dict

100%|██████████| 100/100 [00:07<00:00, 13.43it/s]


{'2012-01': {'num_of_cluseer': 3,
  'random_state': 14,
  'model': KMeans(max_iter=500, n_clusters=3, n_init=1, random_state=14)},
 '2013-01': {'num_of_cluseer': 3,
  'random_state': 9,
  'model': KMeans(max_iter=500, n_clusters=3, n_init=1, random_state=9)},
 '2014-01': {'num_of_cluseer': 3,
  'random_state': 35,
  'model': KMeans(max_iter=500, n_clusters=3, n_init=1, random_state=35)},
 '2015-01': {'num_of_cluseer': 3,
  'random_state': 10,
  'model': KMeans(max_iter=500, n_clusters=3, n_init=1, random_state=10)},
 '2016-01': {'num_of_cluseer': 3,
  'random_state': 0,
  'model': KMeans(max_iter=500, n_clusters=3, n_init=1, random_state=0)},
 '2017-01': {'num_of_cluseer': 3,
  'random_state': 0,
  'model': KMeans(max_iter=500, n_clusters=3, n_init=1, random_state=0)},
 '2018-01': {'num_of_cluseer': 3,
  'random_state': 31,
  'model': KMeans(max_iter=500, n_clusters=3, n_init=1, random_state=31)},
 '2019-01': {'num_of_cluseer': 3,
  'random_state': 29,
  'model': KMeans(max_iter=500, n

## 모델기록


In [19]:
size =evaluate.eval_cluster_size(model_save_dict)
size.to_csv("{}_클러스터사이즈.csv".format(model_name))

print(size.iloc[:-1,:].max().max())
print(size.iloc[:-1,:].min().min())
size

758.0
501.0


,2012-01,2013-01,2014-01,2015-01,2016-01,2017-01,2018-01,2019-01,2020-01,2021-01,2022-01
0,519.0,596.0,568.0,536.0,623.0,592.0,569.0,612.0,541.0,725.0,758.0
1,574.0,505.0,502.0,619.0,606.0,528.0,608.0,617.0,709.0,597.0,724.0
2,501.0,504.0,550.0,504.0,502.0,667.0,688.0,690.0,727.0,704.0,574.0
기업수,1594.0,1605.0,1620.0,1659.0,1731.0,1787.0,1865.0,1919.0,1977.0,2026.0,2056.0


In [20]:
eval_one, eval_one_original = evaluate.eval_cluster_result(model_save_dict, raw_data = scaled_together, label_data=label_data_pp_man, freq=freq)
eval_one.to_csv("{}_관리종목.csv".format(model_name))
eval_one_original.to_csv("{}_관리종목전체대비.csv".format(model_name))

In [21]:
eval_two,eval_two_original = evaluate.eval_cluster_result(model_save_dict, raw_data = scaled_together, label_data=label_data_pp_war, freq=freq)
eval_two.to_csv("{}_투자환기종목.csv".format(model_name))
eval_two_original.to_csv("{}_투자환기종목전체대비.csv".format(model_name))

In [22]:
# 횡령인 경우
eval_three, eval_three_original = evaluate.eval_cluster_result(model_save_dict, raw_data = scaled_together, label_data=label_data_pp_emb, freq=freq)
eval_three.to_csv("{}_횡령배임혐의.csv".format(model_name))
eval_three_original.to_csv("{}_횡령배임혐의전체대비.csv".format(model_name))

In [23]:
print(evaluate.calculate_catch_probability(eval_one, eval_two, None))#, eval_three)
print(evaluate.calculate_match_probability(eval_one, eval_two,None))# eval_three)
evaluate.calculate_catch_probability(eval_one_original, eval_two_original, None)#eval_three_original)

0.8856525278614995
1.0


0.10467845191328387

In [24]:
def pct(df):
    return df / np.sum(df)

dist = evaluate.eval_cluster_distance(eval_one, scaled_together, model_save_dict)
dist = dist.apply(pct, axis=1)
dist2= dist.mean(axis=0).sort_values(ascending=False)

dist.to_csv("{}_거리분해.csv".format(model_name))
dist2.to_csv("{}_거리분해평균.csv".format(model_name))

dist2

매출총이익                             0.165045
총자본                               0.142014
영업이익                              0.131190
당기순이익                             0.106276
매출채권                              0.106216
판매 및 관리비                          0.103632
영업현금흐름                            0.083292
최대주주비율                            0.068776
BW금액+CB금액+유상증자금액                  0.023240
결산기간_주가변동률                        0.020627
차입금의존도                            0.019536
경영권분쟁소송+불성실공시법인지정카운트+조회공시요구카운트    0.014013
최대주주비율_변동성                        0.010761
결산기간_거래량증가                        0.005382
dtype: float64

In [25]:
pre, pre_original = evaluate.predict_cluster_result(model_save_dict, scaled_together,label_data_pp_man, "2012-01", freq=freq)
pre

c:\Jehyeon\2022_Winter_KRX\KRX\Model\krx_fr\cluster\optimize.py:263: RuntimeWarning: invalid value encountered in double_scalars
  cluster_accuracy[key] = cluster_accuracy[key]/value_sum


,2013-01예측,2014-01예측,2015-01예측,2016-01예측,2017-01예측,2018-01예측,2019-01예측,2020-01예측,2021-01예측
0,0.914286,0.069767,0.071429,0.906977,0.134615,0.825581,0.000000,0.011494,0.030769
1,0.000000,0.069767,0.023810,0.069767,0.826923,0.046512,0.955056,0.862069,0.876923
2,0.085714,0.860465,0.904762,0.023256,0.038462,0.127907,0.044944,0.126437,0.092308


In [26]:
pre, pre_original = evaluate.predict_cluster_result(model_save_dict, scaled_together,label_data_pp_man, "2012-01", freq=freq)
pre

c:\Jehyeon\2022_Winter_KRX\KRX\Model\krx_fr\cluster\optimize.py:263: RuntimeWarning: invalid value encountered in double_scalars
  cluster_accuracy[key] = cluster_accuracy[key]/value_sum


,2013-01예측,2014-01예측,2015-01예측,2016-01예측,2017-01예측,2018-01예측,2019-01예측,2020-01예측,2021-01예측
0,0.914286,0.069767,0.071429,0.906977,0.134615,0.825581,0.000000,0.011494,0.030769
1,0.000000,0.069767,0.023810,0.069767,0.826923,0.046512,0.955056,0.862069,0.876923
2,0.085714,0.860465,0.904762,0.023256,0.038462,0.127907,0.044944,0.126437,0.092308


In [27]:
# 그림뽑기

test = MyKmeans.visualize_3d_with_answer(scaled_together["2021-01"],model_save_dict["2021-01"]["model"], answer_data=label_data_pp_man["2021-01"], fig_title="{}_관리종목".format(model_name))

In [28]:
MyKmeans.visualize_3d_with_answer(scaled_together["2021-01"],model_save_dict["2021-01"]["model"], answer_data=label_data_pp_war["2021-01"], fig_title="{}_투자주의환기".format(model_name))

,PC1,PC2,PC3,label,answer
A000020,0.769611,-0.124326,0.263744,1,2.5
A000040,-0.293869,0.283699,-0.388314,0,2.5
A000050,0.775433,-0.004073,0.114882,1,2.5
A000070,1.202463,0.218727,0.066567,1,2.5
A000080,1.158450,0.233914,0.160075,1,2.5
...,...,...,...,...,...
A353810,0.081648,0.022000,0.015594,2,2.5
A354200,-0.951416,-0.120624,0.237982,0,2.5
A357780,0.928167,-0.328154,-0.433330,1,2.5
A363280,0.501468,-0.019126,-0.313810,1,2.5
